# Yelp Data Set Challenge Data and Lambda Unit 2 Project

### Use yelps data to build a prediction model

## Step 1: Data Exploration
### load, clean and explore the data


### Imports

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import files

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
source = '../yelp_data_applied_modeling/'

In [4]:
files = [
    'business.json',
    'checkin.json',
    'review.json',
    'tip.json',
    'user.json',
    'photo.json'
]

### Read JSON files, print out the shapes and first few rows of the Business DF since it will be the first one cleaned

In [5]:
business = pd.read_json(source + files[0], lines=True)

In [6]:
#tips = pd.read_json(source + files[3], lines=True)

In [7]:
#check_in = pd.read_json(source + files[1], lines=True)

In [8]:
#df_chunk = pd.read_json(source + files[5], lines=True, chunksize=500000)
#chunk_list = []

#for chunk in df_chunk:
    #chunk_list.append(chunk)
    
#photos = pd.concat(chunk_list)

In [9]:
#df_chunk = pd.read_json(source + files[4], lines=True, chunksize=500000)
#chunk_list = []

#for chunk in df_chunk:
   # chunk_list.append(chunk)
    
#users = pd.concat(chunk_list)

In [10]:
df_chunk = pd.read_json(source + files[2], lines=True, chunksize=500000)
chunk_list = []

for chunk in df_chunk:
    chunk_list.append(chunk)
    
reviews = pd.concat(chunk_list)

In [11]:
print(f'Business df shape: {business.shape}')
# print(f'Check ins df shape: {check_in.shape}')
print(f'Reviews df shape: {reviews.shape}')
# print(f'Tips df shape: {tips.shape}')
# print(f'Users df shape: {users.shape}')
# print(f'Photos df shape: {photos.shape}')

Business df shape: (192609, 14)
Reviews df shape: (6685900, 9)


In [12]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


### unpack Business DataFrame -- attributes, hours, and categories

In [13]:
def unpack(df, col):
    df = df.copy()
    df_col = df[col].apply(pd.Series)
    df = pd.concat([df, df_col], axis=1)
    return df

def str_to_dict(df, col):
    df = df.copy()
    x = df[col]
    
    for i in range(len(df)):
        if type(x[i]) == str:
            x[i] = eval(x[i])
            
    df_col = df[col].apply(pd.Series)
    df = pd.concat([df, df_col], axis=1)
    return df

In [14]:
business = unpack(business, 'attributes')
business = unpack(business, 'hours')

In [15]:
categories = [
    'Ambience',
    'BusinessParking',
    'GoodForMeal',
    'Music',
    'BestNights'
]

for cat in categories:
    business = str_to_dict(business, cat)

C:\Users\andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [16]:
business = business.drop(columns=[
    0,
    'attributes',
    'hours',
    'BikeParking',
    'Ambience',
    'GoodForMeal',
    'Music',
    'BestNights'
])

In [17]:
business.shape

(192609, 67)

In [18]:
business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'categories', 'GoodForKids', 'RestaurantsReservations',
       'BusinessParking', 'Caters', 'NoiseLevel', 'RestaurantsTableService',
       'RestaurantsTakeOut', 'RestaurantsPriceRange2', 'OutdoorSeating',
       'HasTV', 'WiFi', 'Alcohol', 'RestaurantsAttire',
       'RestaurantsGoodForGroups', 'RestaurantsDelivery',
       'BusinessAcceptsCreditCards', 'BusinessAcceptsBitcoin',
       'ByAppointmentOnly', 'AcceptsInsurance', 'GoodForDancing', 'CoatCheck',
       'HappyHour', 'WheelchairAccessible', 'DogsAllowed', 'BYOBCorkage',
       'DriveThru', 'Smoking', 'AgesAllowed', 'HairSpecializesIn', 'Corkage',
       'BYOB', 'DietaryRestrictions', 'Open24Hours',
       'RestaurantsCounterService', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday', 'Friday', 'Saturday', 'Sunday', 'casual', 'classy', 'divey',
       'hipster', 'intimate', 'roma

In [19]:
categories = business['categories'].str.split(',', expand=True).rename(columns=lambda x: f'category {x+1}')

In [20]:
business = pd.concat([business, categories], axis=1).drop(columns='categories')

In [21]:
business.shape

(192609, 103)

In [22]:
cols_to_drop = [f'category {x+1}' for x in range(5,37)]

business = business.drop(columns=cols_to_drop)

In [23]:
business['density'] = business['postal_code'].groupby(business['postal_code']).transform('size')

In [24]:
business.isnull().sum()

business_id                        0
name                               0
address                            0
city                               0
state                              0
postal_code                        0
latitude                           0
longitude                          0
stars                              0
review_count                       0
is_open                            0
GoodForKids                   126299
RestaurantsReservations       140322
BusinessParking                91131
Caters                        151981
NoiseLevel                    148730
RestaurantsTableService       175436
RestaurantsTakeOut            130532
RestaurantsPriceRange2         84430
OutdoorSeating                137786
HasTV                         144511
WiFi                          142545
Alcohol                       144146
RestaurantsAttire             143970
RestaurantsGoodForGroups      137891
RestaurantsDelivery           140087
BusinessAcceptsCreditCards     79476
B

In [25]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,GoodForKids,RestaurantsReservations,BusinessParking,Caters,NoiseLevel,RestaurantsTableService,RestaurantsTakeOut,RestaurantsPriceRange2,OutdoorSeating,HasTV,WiFi,Alcohol,RestaurantsAttire,RestaurantsGoodForGroups,RestaurantsDelivery,BusinessAcceptsCreditCards,BusinessAcceptsBitcoin,ByAppointmentOnly,AcceptsInsurance,GoodForDancing,CoatCheck,HappyHour,WheelchairAccessible,DogsAllowed,BYOBCorkage,DriveThru,Smoking,AgesAllowed,HairSpecializesIn,Corkage,BYOB,DietaryRestrictions,Open24Hours,RestaurantsCounterService,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,casual,classy,divey,hipster,intimate,romantic,touristy,trendy,upscale,garage,lot,street,valet,validated,category 1,category 2,category 3,category 4,category 5,density
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf,Active Life,None,None,None,1430
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,True,True,"{'garage': False, 'street': False, 'validated'...",True,u'loud',True,True,2,False,False,u'no',u'full_bar',u'casual',True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:0-0:0,9:0-0:0,9:0-0:0,9:0-0:0,9:0-1:0,9:0-1:0,9:0-0:0,True,False,False,False,False,False,False,False,False,False,True,False,False,False,Specialty Food,Restaurants,Dim Sum,Imported Food,Food,8
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,True,True,"{'garage': False, 'street': False, 'validated'...",False,u'average',True,True,2,False,True,u'no',u'beer_and_wine','casual',True,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17:30-21:30,NaN,17:30-21:30,17:30-21:30,17:30-22:0,17:30-22:0,17:30-21:0,True,False,False,False,False,False,False,False,False,False,True,False,False,False,Sushi Bars,Restaurants,Japanese,None,None,347
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Insurance,Financial Services,None,None,None,518
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:0-23:0,7:0-23:0,7:0-23:0,7:0-23:0,7:0-23:0,7:0-23:0,7:0-23:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Plumbing,Shopping,Local Services,Home Services,Kitchen & Bath,531


## Merge Reviews and Businesses together, add Sentiment feature

In [26]:
business.shape

(192609, 72)

In [27]:
reviews.head(1)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36


In [28]:
reviews.shape

(6685900, 9)

In [29]:
reviews = pd.merge(reviews, business, how='left', on='business_id')
reviews.shape

(6685900, 80)

In [30]:
reviews = reviews.rename(
    columns={'stars_x': 'review_stars',
             'stars_y': 'business_stars'}
)

In [31]:
reviews.loc[reviews['review_stars'] <= 3, 'sentiment'] = 'neg'
reviews.loc[reviews['review_stars'] > 3, 'sentiment'] = 'pos'

In [32]:
reviews.shape

(6685900, 81)

In [33]:
reviews['review_stars'].value_counts(normalize=True)

5    0.438697
4    0.219714
1    0.149891
3    0.110573
2    0.081125
Name: review_stars, dtype: float64

In [34]:
reviews['business_stars'].value_counts(normalize=True)

4.0    0.314639
3.5    0.220587
4.5    0.183279
3.0    0.124378
2.5    0.061175
5.0    0.052535
2.0    0.026905
1.5    0.012101
1.0    0.004403
Name: business_stars, dtype: float64

In [35]:
reviews['sentiment'].value_counts(normalize=True)

pos    0.658411
neg    0.341589
Name: sentiment, dtype: float64

In [36]:
reviews['area_avg_stars'] = reviews['review_stars'].groupby(reviews['postal_code']).transform('mean')
reviews['area_median_review_count'] = reviews['review_count'].groupby(reviews['postal_code']).transform('median')

In [37]:
reviews['bus_dif_avg_stars'] = (reviews['business_stars'] - reviews['area_avg_stars'])
reviews['count_dif_median'] = (reviews['review_count'] - reviews['area_median_review_count'])

In [38]:
reviews.shape

(6685900, 85)

In [39]:
reviews.head()

,review_id,user_id,business_id,review_stars,useful,funny,cool,text,date,name,address,city,state,postal_code,latitude,longitude,business_stars,review_count,is_open,GoodForKids,RestaurantsReservations,BusinessParking,Caters,NoiseLevel,RestaurantsTableService,RestaurantsTakeOut,RestaurantsPriceRange2,OutdoorSeating,HasTV,WiFi,Alcohol,RestaurantsAttire,RestaurantsGoodForGroups,RestaurantsDelivery,BusinessAcceptsCreditCards,BusinessAcceptsBitcoin,ByAppointmentOnly,AcceptsInsurance,GoodForDancing,CoatCheck,HappyHour,WheelchairAccessible,DogsAllowed,BYOBCorkage,DriveThru,Smoking,AgesAllowed,HairSpecializesIn,Corkage,BYOB,DietaryRestrictions,Open24Hours,RestaurantsCounterService,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,casual,classy,divey,hipster,intimate,romantic,touristy,trendy,upscale,garage,lot,street,valet,validated,category 1,category 2,category 3,category 4,category 5,density,sentiment,area_avg_stars,area_median_review_count,bus_dif_avg_stars,count_dif_median
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,MountainView Hospital,3100 N Tenaya Way,Las Vegas,NV,89128,36.215546,-115.248798,2.5,238,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fitness & Instruction,Doctors,Health & Medical,Active Life,Gyms,871,neg,3.766359,127.0,-1.266359,111.0
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,Kelly Cardenas Salon - Hard Rock Hotel & Casino,4455 Paradise Rd,Las Vegas,NV,89169,36.110083,-115.153871,4.5,66,1,True,NaN,"{'garage': True, 'street': False, 'validated':...",NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:0-17:0,10:0-19:0,10:0-19:0,10:0-19:0,10:0-19:0,10:0-19:0,10:0-17:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,False,False,Beauty & Spas,Hair Stylists,Makeup Artists,Hair Salons,Hair Extensions,534,pos,3.771289,521.0,0.728711,-455.0
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03,J. Philipp Centers for Family and Cosmetic Den...,"3230 S Gilbert Rd, Ste 4",Chandler,AZ,85286,33.259702,-111.790203,3.5,39,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0:0-0:0,7:0-17:0,NaN,7:0-17:0,7:0-15:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Health & Medical,Cosmetic Dentists,Orthodontists,General Dentistry,Dentists,541,pos,3.607090,97.0,-0.107090,-58.0
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,Regal Beagle Pub,180 94 Ave SE,Calgary,AB,T2J 3G8,50.969721,-114.070115,3.0,13,1,NaN,False,NaN,NaN,NaN,NaN,NaN,2,False,True,NaN,'full_bar',NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,Bars,Pubs,Nightlife,Tapas Bars,Restaurants,7,pos,4.079365,89.0,-1.079365,-76.0
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,White Teeth AZ,"4160 N Craftsman Ct, Ste 202",Scottsdale,AZ,85251,33.496713,-111.927958,4.5,75,1,NaN,NaN,"{'garage': False, 'street': True, 'validated':...",NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,True,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:0-19:0,10:0-19:0,10:0-19:0,10:0-19:0,10:0-19:0,10:0-19:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [44]:
review_text = reviews[['review_id','text', 'review_stars', 'sentiment']]

In [45]:
review_text.head()

,review_id,text,review_stars,sentiment
0,Q1sbwvVQXV2734tPgoKj4Q,Total bill for this horrible service? Over $8G...,1,neg
1,GJXCdrto3ASJOqKeVWPi6Q,I *adore* Travis at the Hard Rock's new Kelly ...,5,pos
2,2TzJjDVDEuAW6MR5Vuc1ug,I have to say that this office really has it t...,5,pos
3,yi0R0Ugj_xUx_Nek0-_Qig,Went in for a lunch. Steak sandwich was delici...,5,pos
4,11a8sVPMUFtaC7_ABRkmtw,Today was my second out of three sessions I ha...,1,neg


In [46]:
# save data frame as csv and download the file
file_name = 'reviews_text.csv'
export_csv = review_text.to_csv(file_name, index=None, header=True)

In [43]:
# save data frame as csv and download the file
file_name = 'reviews_with_business.csv'
export_csv = reviews.to_csv(file_name, index=None, header=True)